importing excel sheet


In [120]:
import pandas as pd 
import numpy as np
import requests
from rich import print_json
from bs4 import BeautifulSoup
import json


df = pd.read_excel('prices3.xlsx')
df["sku"] = df["sku"].astype(str)

df.head()

,sku,special price
0,ORL-00006341,6.600
1,ORL-00006341,6.601
2,I-00000190608,15.000
3,I-00000190608,15.001
4,ORL-00006239,22.500


In [121]:
def fetchSlug(sku): 
    url = "https://ksa-api.boutiqaat.com/searchplus/rest/V2/global/suggest"
    headers = {
        "accept": "application/json, text/plain, */*",
        "accept-language": "en",
        "content-type": "application/json",
        "origin": "https://www.boutiqaat.com",
        "param": '{"gender":"4194","app_version":"8.0.0","platform":"web","deviceId":"1759937968632-266715","exp_list":{"exp-55139":"default","exp-36818":"disabled","exp-36740":"default"}}',
        "priority": "u=1, i",
        "referer": "https://www.boutiqaat.com/",
        "sec-ch-ua": '"Google Chrome";v="141", "Not?A_Brand";v="8", "Chromium";v="141"',
        "sec-ch-ua-mobile": "?0",
        "sec-ch-ua-platform": '"Linux"',
        "sec-fetch-dest": "empty",
        "sec-fetch-mode": "cors",
        "sec-fetch-site": "same-site",
        "user-agent": "Mozilla/5.0 (X11; Linux x86_64) AppleWebKit/537.36 (KHTML, like Gecko) Chrome/141.0.0.0 Safari/537.36"
    }
    payload = {
        "productId": "",
        "tvId": "",
        "slug": "",
        "celebrityId": "",
        "categoryId": "",
        "brandId": "",
        "celebrityIds": "",
        "categoryIds": "",
        "brandIds": "",
        "optionId": "",
        "attributeId": "",
        "countryCodeAndLanguage": "kw_en",
        "numberOfRecords": 20,
        "newOrAll": "",
        "featuredOnly": None,
        "sortKey": "",
        "sortDirection": "",
        "searchString": sku
    }
    response = requests.post(url, headers=headers, json=payload)
    # print(response.status_code)
    # print_json(data=response.json())
    data = response.json()
    return data["data"][0]["data"][0]["slug"]

fetchSlug("124374")

'black-mamool-36gm/p/'

In [122]:
def fetchHtmlPage(slug):
    url = (f"https://www.boutiqaat.com/en-kw/men/{slug}")

    response = requests.get(url)
    html = response.text
    soup = BeautifulSoup(html, "html.parser")


    script_tag = soup.find("script", {"id": "__NEXT_DATA__"})
    data = json.loads(script_tag.string)

    final_price = data["props"]["pageProps"]["response"][0]["final_price_with_tax"]
    return final_price
fetchHtmlPage('black-mamool-36gm/p/')

'9.000'

In [123]:
def fetchPrice(sku):
    slug = fetchSlug(sku)
    web_price = fetchHtmlPage(slug)
    return web_price


In [124]:
for index, row in df.iterrows():
    sku = row["sku"]
    
    original_price = row["special price"]
    
    web_price = fetchPrice(sku)
    if float(web_price) != float(original_price):
        print(f"SKU {sku}: Excel price = {original_price}, API price = {web_price} → Mismatch!")
        df.loc[index, "match"] = "item mismatch"
    else:
        print(f"SKU {sku}: Prices match.")
        df.loc[index, "match"] = "match"
df.to_excel("comparison_results.xlsx", index=False)


SKU ORL-00006341: Prices match.
SKU ORL-00006341: Excel price = 6.601, API price = 6.600 → Mismatch!
SKU I-00000190608: Prices match.
SKU I-00000190608: Excel price = 15.001, API price = 15.000 → Mismatch!
SKU ORL-00006239: Prices match.
SKU ORL-00006239: Excel price = 22.501, API price = 22.500 → Mismatch!
SKU ORL-00004724: Prices match.
SKU ORL-00004724: Excel price = 17.131, API price = 17.130 → Mismatch!
SKU MU-00022119: Prices match.
SKU MU-00022119: Excel price = 8.501, API price = 8.500 → Mismatch!
SKU ORL-00003818: Prices match.
SKU ORL-00003818: Excel price = 30.001, API price = 30.000 → Mismatch!
SKU ORL-00007221: Prices match.
SKU ORL-00007221: Excel price = 5.721, API price = 5.720 → Mismatch!
SKU Conf-589239: Prices match.
SKU Conf-589239: Excel price = 12.751, API price = 12.750 → Mismatch!
SKU SC-00011934: Prices match.
SKU SC-00011934: Excel price = 11.701, API price = 11.700 → Mismatch!
SKU Conf-590088: Prices match.
SKU Conf-590088: Excel price = 11.251, API price = 1